# Lid-driven Cavity

[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-pbs/PhiFlow/blob/develop/examples/grids/Lid_Driven_Cavity.ipynb)

This notebook implements the popular lid-driven cavity fluid simulation.
Fluid inside a closed box is forced at the top boundary which has a constant right-pointing velocity.

In [1]:
%pip install phiflow
from phi.jax.flow import *
# from phi.flow import *  # If JAX is not installed. You can use phi.torch or phi.tf as well.

We define our fluid simulation consisting of advection, diffusion and incompressibility.

In [8]:
@jit_compile
def step(v, p, dt=1., viscosity=.1):
    v = advect.semi_lagrangian(v, v, dt)
    v = diffuse.explicit(v, viscosity, dt)
    v, p = fluid.make_incompressible(v, solve=Solve(x0=p))
    return v, p

Next, we define the boundary and run the simulation.

In [12]:
boundary = {'x': 0, 'y-': 0, 'y+': vec(x=1, y=0)}
v0 = StaggeredGrid(0, boundary, x=50, y=32)
v_trj, p_trj = iterate(step, batch(time=300), v0, None)

In [14]:
frames = v_trj.time[::8]
plot(frames, animate='time')